In [4]:
import ee
import geemap
import numpy as np
import sys
import geopandas as gpd
import pandas as pd
sys.path.insert(1, '/Users/gopal/Projects/ml/downloadGEErasters')
sys.path.insert(1, '/Users/gopal/Projects/sasia/src/proc/')
import rs
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
# from rs import prep_tm5_ic, prep_etm7_ic, prep_oli8_ic, prep_landsat_clouds

In [5]:
ee.Initialize()

In [6]:
ee_geometry = ee.FeatureCollection('users/gopalpenny/arkavathytanks/ArkavatiSubbasins')
tg_halli = ee_geometry.filter(ee.Filter.eq('Subcatch', 'TG_Halli'))
tg_halli.aggregate_array('Subcatch').getInfo()

['TG_Halli']

In [7]:

tm5 = rs.prep_tm5_ic(ee_geometry).filterDate('2014-06-01','2015-06-01')
etm7 = rs.prep_etm7_ic(ee_geometry).filterDate('2014-06-01','2015-06-01')
oli8 = rs.prep_oli8_ic(tg_halli).filterDate('2014-06-01','2015-06-01')

landsat = tm5.merge(etm7).merge(oli8) \
    .filter(ee.Filter.lt('CLOUD_COVER', 50))

landsat_clouds = rs.prep_landsat_clouds(landsat)

In [8]:
# landsat = landsat #\
  # .filterDate('2014-11-01','2015-02-01') #\
  # .filter(ee.Filter.inList('DATE_ACQUIRED', ee.List(['2014-11-10','2015-01-13'])))

# landsat_clouds.aggregate_array('DATE_ACQUIRED').getInfo()

In [9]:
samp_pts_full = gpd.read_file('../../spatial/unmod/training/Landuse_village_datapoints_15_01_2015.shp')
samp_pts_full['id'] = np.arange(0, len(samp_pts_full))

samp_pts_df = samp_pts_full.drop(columns=['geometry','Longitude','Latitude','GD_COLLECT','Unique_ID'])
# samp_pts_ids['id'] = np.arange(0, len(samp_pts_ids))
samp_pts_df.to_csv('../../data/format/training_pt_ids.csv', index=False)

In [17]:

samp_pts = samp_pts_full[['id', 'geometry']]
samp_pts


,id,geometry
0,0,POINT (77.30060 13.19500)
1,1,POINT (77.30160 13.19530)
2,2,POINT (77.30250 13.19580)
3,3,POINT (77.30320 13.19590)
4,4,POINT (77.30330 13.19530)
...,...,...
614,614,POINT (77.27383 13.27420)
615,615,POINT (77.27198 13.26347)
616,616,POINT (77.26981 13.24523)
617,617,POINT (77.28504 13.22306)


In [18]:
samp_pts_ee = geemap.geopandas_to_ee(samp_pts)

# oli8.select([0]).projection()



In [19]:
# print(rs.get_pixel_ts_allbands.__doc__)
training_ts = rs.get_pixel_ts_allbands(samp_pts_ee, landsat_clouds, 'DATE_ACQUIRED', scale = 30)
# import inspect
# print(inspect.getsource(rs.get_pixel_ts_allbands))

# crops_task = ee.batch.Export.image.toDrive(
#     image = crops_bands,
#     description = 'crops_bands',
#     folder = 'classy_downloads',
#     fileNamePrefix = 'karur_crops_2015_2021',
#     region = ee_geometry_box,
#     scale = 30)

# # crops_task.start()
export_ts = ee.batch.Export.table.toDrive(
    collection = training_ts,
    description = 'training_ts',
    folder = 'gee_sasia',
    fileNamePrefix = 'landsat_for_lulc_2014_2015',
    fileFormat = 'CSV')

export_ts.start()

In [20]:
ee.batch.Task.list()

[<Task Z35FMMQUUQUCO7BXPGVQTXCP EXPORT_FEATURES: training_ts (READY)>,
 <Task 2VX2HDDDGHADYNXN63F2BYSL EXPORT_FEATURES: training_ts (COMPLETED)>,
 <Task 6JV7R35NSF6CDMQLBBOX6OJW EXPORT_IMAGE: crops_bands (COMPLETED)>]

## Import the GEE output to repository

In [24]:
landsat_data_gee = pd.read_csv(
    '../../data/gdrive_gee_sasia/landsat_for_lulc_2014_2015.csv')

landsat_data = landsat_data[['id', 'image_id', 'blue', 'green', 'red',
                             'nir', 'swir1', 'swir2', 'clouds', 'shadows', 'clouds_shadows']]
landsat_data.to_csv('../../data/format/training_landsat_2014_2015.csv', index=False)


In [14]:
m = geemap.Map()
m.addLayer(oli8.filterDate('2014-11-01','2014-12-01').first(),{'bands':['nir','red','green'], 'max': 30000},'oli8 2014')
m.addLayer(oli8.filterDate('2015-01-01','2015-12-01').first(),{'bands':['nir','red','green'], 'max': 30000},'oli8 2015')
m.addLayer(ee.Image().paint(ee_geometry,1,1),{'palette':'white'},'ark')
m.addLayerControl()
m.centerObject(ee_geometry, 10)
m

Map(center=[12.84852761341192, 77.44092789490166], controls=(WidgetControl(options=['position', 'transparent_b…